# HOME-WORK SEVEN
## Taruna Mangar

# TOP QUERY (1)

## Exercise

- Write a query against the dbo.Orders table that returns the
- total quantities for each:
- employee, customer, and order year
- employee and order year
- customer and order year
- Include a result column in the output that uniquely identifies
- the grouping set with which the current row is associated
- Tables involved: TSQLV4 database, dbo.Orders table

**PROPOSITION**: Create a query to sum order quantities in the dbo.Orders table across dimensions of EmployeeId, CustomerId, and OrderYear, using `GROUPING SETS` for detailed combinations: by both employee and customer annually, by employee across years, and by customer across years. Incorporate `GROUPING_ID` to uniquely identify each grouping set, allowing for an intricate analysis that clearly separates aggregation levels.

**DATABASE**: Northwinds2022TSQLV7 Database

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, OrderDate, Qty

**PREDICATE**: This query leverages `GROUPING SETS` and `GROUPING_ID` for detailed multi-dimensional analysis of order volumes by employee, customer, and year in the Northwinds2022TSQLV7 database. It aggregates data across various combinations, offering a nuanced view of sales trends, employee performance, and customer engagement. This approach provides valuable insights for strategic planning and performance evaluation.

**WHAT'S SPECIAL**: Using `GROUPING SETS` and `GROUPING_ID`, this query uniquely aggregates order quantities across different dimensions—employee, customer, and year. It creates a comprehensive analysis that spans from specific interactions to broader summaries, all within a single execution. This showcases SQL's ability to efficiently present data from multiple angles, enabling detailed strategic insights.

In [ ]:
Use TSQLV4
SELECT
  GROUPING_ID(empid, custid, YEAR(Orderdate)) AS groupingset,
  empid, custid, YEAR(Orderdate) AS orderyear, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY
  GROUPING SETS
  (
    (empid, custid, YEAR(orderdate)),
    (empid, YEAR(orderdate)),
    (custid, YEAR(orderdate))
  );

Use Northwinds2022TSQLV7
SELECT
  GROUPING_ID(EmployeeId, CustomerId, YEAR(Orderdate)) AS GroupingSet,
  EmployeeId, CustomerId, YEAR(Orderdate) AS OrderYear, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY
  GROUPING SETS
  (
    (EmployeeId, CustomerId, YEAR(OrderDate)),
    (EmployeeId, YEAR(OrderDate)),
    (CustomerId, YEAR(orderdate))
  );

# Offset Window Functions

# TOP QUERY (2)

### Query #1

**PROPOSITION**: Write a query to select the CustomerId, OrderId, and Val, along with the previous and next values of Val for each order per customer. The previous value (PrevVal) should be determined by looking at the preceding order for the same customer, and the next value (NextVal) should be determined by looking at the subsequent order for the same customer. This should be accomplished using window functions.

**DATABASE**: Northwinds2022TSQLV7 Database

**VIEW**: Sales.OrderValues

**ATTRIBUTES**: CustomerId, OrderId, OrderDate, Val

**PREDICATE**: This query uses `Sales.OrderValues` to track order value changes over time for each customer by employing the `LAG` and `LEAD` window functions. It calculates previous and next order values (PrevVal and NextVal) by partitioning orders by `CustomerId` and ordering them by `OrderDate` and `OrderId`. This approach ensures each order's value is contextualized within its customer's purchase history, providing a sequential view of value changes.

**WHAT'S SPECIAL**: Demonstrating SQL's window functions, `LAG` and `LEAD`, this query offers insights into the sales dynamics of the database by analyzing order values over time for each customer. Partitioning by `CustomerId` and sequencing by `OrderDate` and `OrderId` allows for detailed examination of value fluctuations across orders. This method efficiently uncovers sales trends and customer behaviors, showcasing the power of SQL for in-depth data analysis and strategic decision-making in a single query execution.

In [ ]:
-- LAG and LEAD
Use TSQLV4
SELECT custid, orderid, val,
  LAG(val)  OVER(PARTITION BY custid
                 ORDER BY orderdate, orderid) AS prevval,
  LEAD(val) OVER(PARTITION BY custid
                 ORDER BY orderdate, orderid) AS nextval
FROM Sales.OrderValues
ORDER BY custid, orderdate, orderid;

Use Northwinds2022TSQLV7
SELECT CustomerId, OrderId, Val,
  LAG(Val)  OVER(PARTITION BY CustomerId
                 ORDER BY OrderDate, OrderId) AS PrevVal,
  LEAD(Val) OVER(PARTITION BY CustomerId
                 ORDER BY OrderDate, OrderId) AS NextVal
FROM Sales.OrderValues
ORDER BY CustomerId, OrderDate, OrderId;

# TOP QUERY (3)
### Query #2

**PROPOSITION**: Develop a query to calculate the running total of Val for each customer's orders, along with the percentage this total represents out of the customer's overall order values. This calculation should be based on the chronological order of their purchases, identified by OrderDate and OrderId.

**DATABASE**: Northwinds2022TSQLV7 Database

**TABLES**: Sales.OrderValues

**COLUMNS**: CustomerId, OrderId, OrderDate, Val

**PREDICATE**: The query calculates a running total and percentage of each order's value for customers from the Sales.OrderValues table, using the SUM window function ordered by OrderDate and OrderId. It assesses the significance of individual orders by comparing their cumulative value to the total value of all orders per customer, providing insights into customer purchasing patterns.

**WHAT'S SPECIAL:** This query uniquely harnesses the `FIRST_VALUE` and `LAST_VALUE` window functions within the Northwinds2022TSQLV7 database to capture the initial and final values of orders for each customer, organized by order date and ID. By partitioning the data by `CustomerId`, it ensures each segment is analyzed independently, offering a clear view from the first to the last transaction. 

In [ ]:
-- FIRST_VALUE and LAST_VALUE
Use TSQLV4
SELECT custid, orderid, val,
  FIRST_VALUE(val) OVER(PARTITION BY custid
                        ORDER BY orderdate, orderid
                        ROWS BETWEEN UNBOUNDED PRECEDING
                                 AND CURRENT ROW) AS firstval,
  LAST_VALUE(val)  OVER(PARTITION BY custid
                        ORDER BY orderdate, orderid
                        ROWS BETWEEN CURRENT ROW
                                 AND UNBOUNDED FOLLOWING) AS lastval
FROM Sales.OrderValues
ORDER BY custid, orderdate, orderid;

Use Northwinds2022TSQLV7
SELECT CustomerId, OrderId, Val,
  FIRST_VALUE(Val) OVER(PARTITION BY CustomerId
                        ORDER BY OrderDate, OrderId
                        ROWS BETWEEN UNBOUNDED PRECEDING
                                 AND CURRENT ROW) AS firstval,
  LAST_VALUE(Val)  OVER(PARTITION BY CustomerId
                        ORDER BY OrderDate, OrderId
                        ROWS BETWEEN CURRENT ROW
                                 AND UNBOUNDED FOLLOWING) AS LastVal
FROM Sales.OrderValues
ORDER BY CustomerId, OrderDate, OrderId;



# Pivoting with the PIVOT Operator

# TOP QUERY (4)

### Query #1
**PROPOSITION**: Construct a query to transform the order data into a pivoted format, showing the total quantity of orders for specific customers (identified as A, B, C, D) per employee. This transformation should result in a table where each row represents an employee and columns A, B, C, and D display the total quantities ordered by customers A, B, C, and D, respectively, as processed by that employee.

**DATABASE**: Northwinds2022TSQLV7 Database

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, Qty

**PREDICATE**: This query rearranges the dbo.Orders table data to analyze the volume of orders processed by each employee for specific customers (A, B, C, and D) using a PIVOT operation. It sums the order quantities (Qty) per customer and pivots these totals into columns by CustomerId, making it easier to see the quantity of orders handled by each employee for these selected customers.

**WHAT'S SPECIAL:** This query distinctively employs a PIVOT operation to transform order quantities from the `dbo.Orders` table into a structured format that aligns employee efforts with specific customers (A, B, C, D). By pivoting on `CustomerId` and summarizing the quantities (`Qty`), it directly maps out how many orders each employee has managed for these selected customers. 

In [ ]:
-- Logical equivalent of previous query using the native PIVOT operator
Use TSQLV4
SELECT empid, A, B, C, D
FROM (SELECT empid, custid, qty
      FROM dbo.Orders) AS D
  PIVOT(SUM(qty) FOR custid IN(A, B, C, D)) AS P;

Use Northwinds2022TSQLV7
SELECT EmployeeId, A, B, C, D
FROM (SELECT EmployeeId, CustomerId, Qty
      FROM dbo.Orders) AS D
  PIVOT(SUM(Qty) FOR CustomerId IN(A, B, C, D)) AS P;



# TOP QUERY (5)
### Query #2
**PROPOSITION**: Create a query that transforms the raw order data into a pivoted table format, focusing on summarizing the quantities of orders managed by each employee for four specific customers labeled as A, B, C, and D. The goal is to pivot the quantities (qty) based on the CustomerId, resulting in a structured table where each row corresponds to an EmployeeId and the columns A, B, C, and D represent the total order quantities for each of these specified customers.

**DATABASE**: Northwinds2022TSQLV7 Database

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, qty

**PREDICATE**: This query seeks to aggregate and pivot the order quantities from the dbo.Orders table to efficiently display how each employee's orders are distributed across customers A, B, C, and D. By leveraging the SQL PIVOT function, it aggregates the qty field for orders corresponding to each of the four specified customers, pivoting on the CustomerId. 

**WHAT'S SPECIAL**:What's special about this query is its ability to transform complex transactional data into a clear, organized format that highlights employee engagement with key customers. By pivoting order quantities into columns per customer, it offers immediate insights into workload distribution and performance, making it an invaluable tool for understanding customer service dynamics and optimizing sales strategies.

In [ ]:
-- Query demonstrating the problem with implicit grouping
Use TSQLV4
SELECT empid, A, B, C, D
FROM dbo.Orders
  PIVOT(SUM(qty) FOR custid IN(A, B, C, D)) AS P;

Use Northwinds2022TSQLV7
SELECT EmployeeId, A, B, C, D
FROM dbo.Orders
  PIVOT(SUM(qty) FOR CustomerId IN(A, B, C, D)) AS P;

### Query #3 
**PROPOSITION**: Create a query in the Northwinds2022TSQLV7 database to calculate how many orders each employee has processed for selected customers (A, B, C, D) from the dbo.Orders table. This involves using conditional aggregation to sum order quantities (`qty`) by employee for these specific customers.

**DATABASE**: Northwinds2022TSQLV7

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, qty

**PREDICATE**: This approach aggregates order quantities by both employee and customer, focusing on customers A, B, C, and D. It cleverly utilizes conditional aggregation to tally orders for each employee-customer combination, providing a nuanced look at order volume and employee engagement with key accounts, all without pivoting. This strategy highlights workload distribution among employees and their interaction with significant customers, offering valuable insights for business strategy and operations.

In [ ]:
-- Logical equivalent of previous query
Use TSQLV4
SELECT empid,
  SUM(CASE WHEN custid = 'A' THEN qty END) AS A,
  SUM(CASE WHEN custid = 'B' THEN qty END) AS B,
  SUM(CASE WHEN custid = 'C' THEN qty END) AS C,
  SUM(CASE WHEN custid = 'D' THEN qty END) AS D  
FROM dbo.Orders
GROUP BY orderid, orderdate, empid;

Use Northwinds2022TSQLV7
SELECT EmployeeId,
  SUM(CASE WHEN CustomerId = 'A' THEN qty END) AS A,
  SUM(CASE WHEN CustomerId = 'B' THEN qty END) AS B,
  SUM(CASE WHEN CustomerId = 'C' THEN qty END) AS C,
  SUM(CASE WHEN CustomerId = 'D' THEN qty END) AS D  
FROM dbo.Orders
GROUP BY OrderId, OrderDate, EmployeeId;

### Query #4
**PROPOSITION**: Craft a query in the Northwinds2022TSQLV7 database to pivot the dbo.Orders table's order quantities, targeting the performance of employees 1, 2, and 3 across all customers. The objective is to organize this data into a clear format where each customer is represented by a row, and the columns labeled [1], [2], and [3] indicate total orders handled by these employees, respectively.

**DATABASE**: Northwinds2022TSQLV7

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, Qty

**PREDICATE**: By leveraging the PIVOT function, this query aims to aggregate order quantities based on specific employees' contributions to serving customers, transforming EmployeeId values into column headers to display the summed quantities directly. This approach not only simplifies viewing the data but also offers a direct measure of how key employees are performing in customer service, providing valuable insights for assessing employee distribution and effectiveness in order fulfillment.

In [ ]:
-- Query against Orders, grouping by customer, pivoting employees,
-- aggregating sum of quantity
Use TSQLV4
SELECT custid, [1], [2], [3]
FROM (SELECT empid, custid, qty
      FROM dbo.Orders) AS D
  PIVOT(SUM(qty) FOR empid IN([1], [2], [3])) AS P;

Use Northwinds2022TSQLV7
SELECT CustomerId, [1], [2], [3]
FROM (SELECT EmployeeId, CustomerId, Qty
      FROM dbo.Orders) AS D
  PIVOT(SUM(Qty) FOR EmployeeId IN([1], [2], [3])) AS P;

# GROUPING and GROUPING_ID Function

### Query #1
**PROPOSITION**: Create a query for the Northwinds2022TSQLV7 database to comprehensively summarize order quantities in the dbo.Orders table by EmployeeId and CustomerId, using the CUBE operator. This will allow for a thorough multidimensional analysis, producing subtotals for each employee and customer, along with a grand total for all orders. 

**DATABASE**: Northwinds2022TSQLV7

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, Qty

**PREDICATE**: Aimed at exploring order quantities across varying dimensions of EmployeeId and CustomerId, this query employs the CUBE operator to cover all possible combinations, enabling both detailed and overarching analyses. It generates both specific subtotals and a comprehensive grand total, offering a deep dive into sales distribution and performance. This approach illuminates patterns and trends, aiding in strategic planning and optimization of sales and customer service efforts.

In [ ]:
Use TSQLV4
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

Use Northwinds2022TSQLV7
SELECT EmployeeId, CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);

### Query #2

**PROPOSITION:** Create a query for the dbo.Orders table in the Northwinds2022TSQLV7 database to aggregate order quantities using the CUBE operator, allowing for a multi-dimensional analysis by EmployeeId and CustomerId. Incorporate the GROUPING function to mark rows with indicators for their aggregation level, distinguishing between detailed entries, subtotals, and the overall total.

**DATABASE**: Northwinds2022TSQLV7

**TABLES**: dbo.Orders

**COLUMNS:** EmployeeId, CustomerId, Qty

**PREDICATE:** This query aggregates orders by every possible EmployeeId and CustomerId combination through the CUBE function, achieving a thorough multidimensional analysis. The GROUPING function adds clarity by signaling whether rows are detailed records, subtotals by employee or customer, or the grand total, using additional columns as flags. This approach delivers a layered view of order data, enhancing insights into sales dynamics and customer relationships.








In [ ]:
Use TSQLV4
SELECT
  GROUPING(empid) AS grpemp,
  GROUPING(custid) AS grpcust,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

Use Northwinds2022TSQLV7
SELECT
  GROUPING(EmployeeId) AS GrpEmp,
  GROUPING(CustomerId) AS GrpCust,
  EmployeeId, CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);

### QUERY #3
**PROPOSITION**: Create a query to sum up order quantities in the dbo.Orders table by both EmployeeId and CustomerId using the CUBE operator for in-depth analysis. Add the GROUPING_ID function to mark each result with a unique code, distinguishing between detailed records, subtotals, and the overall total.

**DATABASE**: Northwinds2022TSQLV7

**TABLES**: dbo.Orders

**COLUMNS**: EmployeeId, CustomerId, Qty

**PREDICATE**: By using the CUBE function, this query aggregates order quantities across every possible combination of EmployeeId and CustomerId, plus a grand total. GROUPING_ID enhances this by giving each result a unique identifier, clarifying whether it's a detailed entry, a subtotal, or a total. This setup provides a comprehensive view of sales by employee and customer.

In [ ]:
Use TSQLV4
SELECT
  GROUPING_ID(empid, custid) AS groupingset,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

Use Northwinds2022TSQLV7
SELECT
  GROUPING_ID(EmployeeId, CustomerId) AS groupingset,
  EmployeeId, CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);
